<a href="https://colab.research.google.com/github/BorisLoveDev/agents-of-uniform-difficulty/blob/main/get_answers_table_and_fit10rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import os
import random
np.random.seed(42)
random.seed(42)

In [2]:
from google.colab import drive
# Подключение Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Загрузка и нормализация данных MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

num_classes = 10

# Добавление измерения канала к данным
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Применение one-hot encoding
y_test_ohe = to_categorical(y_test, num_classes)
y_train_ohe = to_categorical(y_train, num_classes)
y_train_ohe.shape

11490434/11490434 [==============================] - 1s 0us/step


(60000, 10)

In [4]:
# Загрузка данных для уровня сложности 0
x_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_0.npy')
y_data_level_0 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_0.npy')

# Загрузка данных для уровня сложности 6
x_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/x_data_difficult_level_6.npy')
y_data_level_6 = np.load('/content/drive/MyDrive/all_difficulty_data_mnist/y_data_difficult_level_6.npy')


In [5]:
# Функция для деформации изображения, как определено ранее
def deform_image_optimized(image, A, B, M=28, NP=5):
    C = M / (NP + 1.0)
    XN, YN = np.zeros(M), np.zeros(M)
    DX, DY = np.linspace(0, M-1, M), np.linspace(0, M-1, M)

    for j in range(NP):
        TXN = (j + 0.5 - np.random.random()) * C
        TYN = (j + 0.5 - np.random.random()) * C
        TDX = (j + 0.5 - np.random.random()) * C
        TDY = (j + 0.5 - np.random.random()) * C
        AXN = B * (1.0 - 2.0 * np.random.random())
        AYN = B * (1.0 - 2.0 * np.random.random())
        ADX = A * (1.0 - 2.0 * np.random.random())
        ADY = A * (1.0 - 2.0 * np.random.random())
        PXN = (0.1 + 0.9 * np.random.random()) * C
        PYN = (0.1 + 0.9 * np.random.random()) * C
        PDX = (0.1 + 0.9 * np.random.random()) * C
        PDY = (0.1 + 0.9 * np.random.random()) * C

        DX += ADX * np.exp(-((DX - TDX) / PDX)**2)
        DY += ADY * np.exp(-((DY - TDY) / PDY)**2)
        XN += AXN * np.exp(-((DX - TXN) / PXN)**2)
        YN += AYN * np.exp(-((DY - TYN) / PYN)**2)

    deformed_image = np.zeros((M, M))
    for j in range(M):
        for i in range(M):
            x_index = int(DX[i] + XN[j])
            y_index = int(DY[j] + YN[i])
            if 0 <= x_index < M and 0 <= y_index < M:
                deformed_image[j, i] = image[y_index, x_index] if y_index < image.shape[0] and x_index < image.shape[1] else 0

    return deformed_image

In [6]:
deform_params = {
    "1": (0.2, 1.0),
    "2": (0.4, 3.0),
    "3": (1, 5.0),
    "4": (3.5, 9.5),
    "5": (1, 5.0),
}

Получение таблицы ответов для 10 акторов

In [7]:
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization, Activation, LeakyReLU, ELU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
import tensorflow as tf


In [8]:

def create_random_cnn_model(input_shape=(28, 28, 1), num_classes=10):
    model = Sequential()

    conv_layer = random.choice([Conv2D, SeparableConv2D])
    filters = random.randint(16, 64)
    kernel_size = random.choice([(3, 3), (5, 5)])
    activation = random.choice(['relu', 'leaky_relu', 'elu', 'selu', 'swish'])

    model.add(conv_layer(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=input_shape))

    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    num_layers = random.randint(1, 3)
    for _ in range(num_layers):
        conv_layer = random.choice([Conv2D, SeparableConv2D])
        filters = random.randint(16, 128)
        kernel_size = random.choice([(3, 3), (5, 5)])
        activation = random.choice(['relu', 'leaky_relu', 'elu', 'selu', 'swish'])

        if activation == 'selu':
            model.add(conv_layer(filters=filters, kernel_size=kernel_size, activation='selu', padding='same'))
        else:
            model.add(conv_layer(filters=filters, kernel_size=kernel_size, padding='same'))
            if activation == 'leaky_relu':
                model.add(LeakyReLU(alpha=0.01))
            elif activation == 'elu':
                model.add(ELU(alpha=1.0))
            elif activation == 'swish':
                model.add(Activation(tf.nn.swish))
            else:  # relu
                model.add(Activation(activation))

        model.add(BatchNormalization())
        if random.random() < 0.5:
            model.add(MaxPooling2D(pool_size=(2, 2)))
        if random.random() < 0.5:
            model.add(Dropout(rate=random.uniform(0.2, 0.5)))

    model.add(GlobalAveragePooling2D())
    model.add(Dense(units=random.randint(64, 128), activation='relu'))
    if random.random() < 0.5:
        model.add(Dropout(rate=random.uniform(0.2, 0.5)))
    model.add(Dense(num_classes, activation='softmax'))

    return model

# Пример использования
model = create_random_cnn_model(input_shape=(28, 28, 1), num_classes=10)
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['accuracy'])


In [9]:
def split_dataset_with_overlap(data, labels, num_chunks, overlap=0.2):
    chunk_size = len(data) // num_chunks
    overlap_size = int(chunk_size * overlap)

    chunks = []
    for i in range(num_chunks):
        start = i * chunk_size - i * overlap_size
        end = start + chunk_size + overlap_size

        if end > len(data):
            end -= len(data)
            chunk_data = np.concatenate((data[start:], data[:end]))
            chunk_labels = np.concatenate((labels[start:], labels[:end]))
        else:
            chunk_data = data[start:end]
            chunk_labels = labels[start:end]

        chunks.append((chunk_data, chunk_labels))

    data_shapes = [chunk[0].shape for chunk in chunks]
    labels_shapes = [chunk[1].shape for chunk in chunks]

    print(f"Формы данных: {data_shapes}")
    print(f"Формы меток: {labels_shapes}")

    return chunks  # Возвращаем список чанков


In [10]:
def defromed_chunk(chunk_data, chunk_labels, deform_params1, deform_params2):
    # Извлекаем параметры деформации
    A1, B1 = deform_params1
    A2, B2 = deform_params2

    augmented_images = []
    augmented_labels = []

    # Применяем деформацию к первым 500 изображениям с параметрами deform_params1
    for image in chunk_data[:500]:
        deformed_image = deform_image_optimized(image.squeeze(), A=A1, B=B1)
        augmented_images.append(deformed_image.reshape(28, 28, 1))
        augmented_labels.append(10)  # Добавляем метку специального класса

    # Применяем деформацию к следующим 500 изображениям с параметрами deform_params2
    for image in chunk_data[500:1000]:
        deformed_image = deform_image_optimized(image.squeeze(), A=A2, B=B2)
        augmented_images.append(deformed_image.reshape(28, 28, 1))
        augmented_labels.append(10)  # Добавляем метку специального класса

    # Преобразуем списки в numpy массивы
    augmented_images = np.array(augmented_images)
    augmented_labels = np.array(augmented_labels)

    # Объединяем исходные данные и метки с дополненными
    chunk_data_augmented = np.concatenate((chunk_data, augmented_images))
    chunk_labels_augmented = np.concatenate((chunk_labels, augmented_labels))

    # Перемешивание дополненных данных и меток
    chunk_data_augmented, chunk_labels_augmented = shuffle(chunk_data_augmented, chunk_labels_augmented, random_state=42)
    # Вывод информации о форме и распределении
    print(f"Чанк данных форма: {chunk_data_augmented.shape}, Чанк меток форма: {chunk_labels_augmented.shape}", end='; ')
    unique, counts = np.unique(chunk_labels_augmented, return_counts=True)
    class_distribution = ', '.join([f"Класс {u}: {c}" for u, c in zip(unique, counts)])
    print(f"Распределение по классам: {class_distribution}")

    return chunk_data_augmented, chunk_labels_augmented

# Для использования этой функции, вызовите её для каждого чанка данных и меток.
# Пример:
# first_chunk_data_augmented, first_chunk_labels_augmented = augment


In [11]:
num_chunks = 10  # Например, разделим на 10 частей
chunks = split_dataset_with_overlap(x_train, y_train, num_chunks, overlap=0.2)

# Определение параметров деформации для чанков
deform_params_first_half = (12.0, 32.0)  # Параметры для первых пяти чанков
deform_params_second_half = (1.0, 5.0)   # Параметры для последних пяти чанков

# Обработка каждого чанка
for i, (chunk_data, chunk_labels) in enumerate(chunks):
    if i < 5:  # Первые пять чанков
        chunk_data_augmented, chunk_labels_augmented = defromed_chunk(
            chunk_data, chunk_labels,
            deform_params1=deform_params_first_half,
            deform_params2=deform_params_first_half)
    else:  # Оставшиеся пять чанков
        chunk_data_augmented, chunk_labels_augmented = defromed_chunk(
            chunk_data, chunk_labels,
            deform_params1=deform_params_first_half,
            deform_params2=deform_params_second_half)

Формы данных: [(7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1), (7200, 28, 28, 1)]
Формы меток: [(7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,), (7200,)]
Чанк данных форма: (8200, 28, 28, 1), Чанк меток форма: (8200,); Распределение по классам: Класс 0: 713, Класс 1: 809, Класс 2: 694, Класс 3: 735, Класс 4: 734, Класс 5: 629, Класс 6: 725, Класс 7: 770, Класс 8: 673, Класс 9: 718, Класс 10: 1000
Чанк данных форма: (8200, 28, 28, 1), Чанк меток форма: (8200,); Распределение по классам: Класс 0: 745, Класс 1: 817, Класс 2: 704, Класс 3: 754, Класс 4: 666, Класс 5: 629, Класс 6: 728, Класс 7: 755, Класс 8: 682, Класс 9: 720, Класс 10: 1000
Чанк данных форма: (8200, 28, 28, 1), Чанк меток форма: (8200,); Распределение по классам: Класс 0: 724, Класс 1: 816, Класс 2: 693, Класс 3: 759, Класс 4: 664, Класс 5: 661, Класс 6: 689, Класс 7: 7

In [12]:
# 1) Создание генератора-аугментатора данных для классификации MNIST
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=False,
    vertical_flip=False
)

In [13]:
# Добавление нулевого класса в тестовые метки
y_test_extended = np.append(y_test_ohe, 0 * np.ones((len(y_test), 1)), axis=1)
y_test_extended[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [ ]:
num_chunks = len(chunks)  # Общее количество чанков
num_classes = 11

# Генерация 10 случайных моделей
models = [create_random_cnn_model(input_shape=x_train.shape[1:], num_classes=num_classes) for _ in range(10)]

# Компиляция моделей
for model in models:
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

save_dir = "/content/drive/MyDrive/all_difficulty_data_mnist/weights_100rcnn_normal_train_data"
os.makedirs(save_dir, exist_ok=True)

for i, (model, (chunk_data, chunk_labels)) in enumerate(zip(models, chunks)):
      # Проверка, была ли модель уже обучена (пропускаем первые 5 модели)
    if i < 5:
        print(f"Пропуск модели {i+1}, так как она уже обучена.")
        continue  # Переход к следующей модели

    # Подготовка валидационного чанка
    val_chunk_index = (i + 1) % num_chunks
    val_chunk_data, val_chunk_labels = chunks[val_chunk_index]
    val_chunk_labels_one_hot = to_categorical(val_chunk_labels, num_classes=num_classes)

    chunk_labels_one_hot = to_categorical(chunk_labels, num_classes=num_classes)
    print(f"Обучение модели {i+1}")
    print(model.summary())
    # Создание каллбеков
    model_checkpoint = ModelCheckpoint(os.path.join(save_dir, f"best_model_{i+1}.h5"), save_best_only=True, monitor='val_accuracy', mode='max')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.001)

    if i % 2 == 0:  # Для четных индексов используем аугментированные данные
        train_generator = datagen.flow(chunk_data, chunk_labels_one_hot, batch_size=32)
        model.fit(train_generator, epochs=100, verbose=1, callbacks=[model_checkpoint, reduce_lr], validation_data=(val_chunk_data, val_chunk_labels_one_hot))
    else:  # Для нечетных индексов используем неаугментированные данные
        model.fit(chunk_data, chunk_labels_one_hot, epochs=100, batch_size=32, verbose=1, callbacks=[model_checkpoint, reduce_lr], validation_data=(val_chunk_data, val_chunk_labels_one_hot))

    # Оценка точности модели на тестовых данных
    print(f"Тестирование модели {i+1}")
    loss, accuracy = model.evaluate(x_test, y_test_extended, verbose=0)
    print(f"Точность модели {i+1} на тестовых данных: {accuracy * 100:.2f}%")

    # Сохранение модели после обучения
    model_path = os.path.join(save_dir, f"model_{i+1}.h5")
    model.save(model_path)
    print(f"Модель {i+1} сохранена в {model_path}")
